In [1]:
import os
import pandas as pd

# csv.gz 파일을 chunk 단위로 읽어옵니다.
def split_read(path, chunksize=100000, condition="") :
    #chunksize = 100000  # 원하는 chunk 사이즈를 설정합니다.
    #condition = '특정열 == "조건"'  # 필터링할 조건을 설정합니다.
    filtered_df = pd.DataFrame() 
    for chunk in pd.read_csv(path, compression='gzip', chunksize=chunksize):
        # 조건에 맞는 행들만 선택하여 필터링합니다.
        if condition == "":
            filtered_df = pd.concat([filtered_df, chunk], ignore_index=True)
        else :
            filtered_chunk = chunk.query(condition)
            filtered_df = pd.concat([filtered_df, filtered_chunk], ignore_index=True)
        # 필터링된 chunk를 처리합니다.
        # 예를 들어, 이 부분에 필터링된 chunk를 다른 데이터프레임에 추가하거나
        # 원하는 작업을 수행할 수 있습니다.
    return filtered_df

# 현재 노트북 파일의 디렉토리를 가져옵니다.
current_directory = os.getcwd()
relative_folder_path = "../2.2/"

base_df = split_read(os.path.join(current_directory, relative_folder_path, "icu/icustays.csv.gz"))

condition = 'subject_id in ['
for i in list(base_df['subject_id'].unique()[:1000]) :
    condition += str(i)
    condition += ', '
condition += ']'


admissions_df = split_read(os.path.join(current_directory, relative_folder_path, "hosp/admissions.csv.gz"), condition = condition)
patients_df = split_read(os.path.join(current_directory, relative_folder_path, "hosp/patients.csv.gz"), condition = condition)
omr_df = split_read(os.path.join(current_directory, relative_folder_path, "hosp/omr.csv.gz"), condition = condition)
omr_df = omr_df.drop_duplicates(subset=['subject_id','chartdate', 'result_name'], keep='last')


diagnois_df = split_read(os.path.join(current_directory, relative_folder_path, "hosp/diagnoses_icd.csv.gz"), condition = condition)
d_diagnois_df = split_read(os.path.join(current_directory, relative_folder_path, "hosp/d_icd_diagnoses.csv.gz"))
diagnois_df = pd.merge(diagnois_df, d_diagnois_df, on = ['icd_code', 'icd_version'], how = 'left')

transfers_df = split_read(os.path.join(current_directory, relative_folder_path, "hosp/transfers.csv.gz"), condition = condition)

icust_df = split_read(os.path.join(current_directory, relative_folder_path, "icu/icustays.csv.gz"), condition = condition)
datetimeevents_df = split_read(os.path.join(current_directory, relative_folder_path, "icu/datetimeevents.csv.gz"), condition = condition)
d_items_df = split_read(os.path.join(current_directory, relative_folder_path, "icu/d_items.csv.gz")) 
datetimeevents_df = pd.merge(datetimeevents_df, d_items_df, on = 'itemid', how = 'left')

procedureevents_df = split_read(os.path.join(current_directory, relative_folder_path, "icu/procedureevents.csv.gz"), condition = condition)
chartevents_df = pd.read_csv(os.path.join(current_directory, relative_folder_path, "icu/chartevents.csv.gz"), compression='gzip', nrows=1000000)
chartevents_df = chartevents_df[chartevents_df['subject_id'].isin(base_df['subject_id'].unique()[:1000])]
chartevents_df = pd.merge(chartevents_df, d_items_df, on = 'itemid', how = 'left')
chartevents_df = chartevents_df.sort_values('charttime')

In [35]:
accum_json = {}
for subject_id in [int(i) for i in patients_df['subject_id'].unique()] :
    accum_json[subject_id] = {}
    accum_json[subject_id]['status'] = {}
    t_patient = patients_df.loc[patients_df['subject_id']==subject_id]
    accum_json[subject_id]['status']['gender'] = t_patient['gender'].iloc[0]
    accum_json[subject_id]['status']['age'] = int(t_patient['anchor_age'].iloc[0])
    accum_json[subject_id]['hadm_id'] = {}
    for hadm_id in [int(j) for j in admissions_df.loc[admissions_df['subject_id']==subject_id]['hadm_id'].unique()] :
        accum_json[subject_id]['hadm_id'][hadm_id] = {}

        #######################환자 기본 상태 관련
        #####Admission 당시 정보 모음
        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info'] = {}
        if chartevents_df.loc[(chartevents_df['subject_id']==subject_id) & (chartevents_df['hadm_id']==hadm_id)].sort_values("charttime").empty :
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['is_icu'] = 'N'
        else :
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['is_icu'] = 'Y'
        t_admission = admissions_df.loc[(admissions_df['subject_id']==subject_id) & (admissions_df['hadm_id']==hadm_id)]
        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['admittime'] = t_admission['admittime'].iloc[0]
        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['dischtime'] = t_admission['dischtime'].iloc[0]

        t_omr = omr_df.loc[(omr_df['subject_id']==subject_id) & (omr_df['chartdate'] <= accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['dischtime'][:10])]
        if len(t_omr) == 0 :
            t_omr = omr_df.loc[(omr_df['subject_id']==subject_id)]
        
        try :
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['bp'] = t_omr.loc[t_omr['result_name']=='Blood Pressure'].tail(1)['result_value'].iloc[0]
        except : 
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['bp'] = ' '

        try :
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['weight'] = t_omr.loc[t_omr['result_name']=='Weight (Lbs)'].tail(1)['result_value'].iloc[0]
        except : 
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['weight'] = ' '

        try :
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['bmi'] = t_omr.loc[t_omr['result_name']=='BMI (kg/m2)'].tail(1)['result_value'].iloc[0]
        except : 
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['bmi'] = ' '

        try :
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['height'] = t_omr.loc[t_omr['result_name']=='Height (Inches)'].tail(1)['result_value'].iloc[0]
        except : 
            accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['height'] = ' '


        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['insurance'] = t_admission['insurance'].iloc[0]
        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['language'] = t_admission['language'].iloc[0]
        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['marital_status'] = t_admission['marital_status'].iloc[0]
        accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['race'] = t_admission['race'].iloc[0]

        ######진단 관련 정보
        accum_json[subject_id]['hadm_id'][hadm_id]['diagnosis'] = {}
        t_diagnosis = diagnois_df.loc[(diagnois_df['subject_id']==subject_id) & (diagnois_df['hadm_id']==hadm_id)]
        accum_json[subject_id]['hadm_id'][hadm_id]['diagnosis']['total_len'] = len(t_diagnosis)
        accum_json[subject_id]['hadm_id'][hadm_id]['diagnosis']['titles'] = []
        for it, row in t_diagnosis.iterrows() :
            accum_json[subject_id]['hadm_id'][hadm_id]['diagnosis']['titles'].append(row['long_title'])
        
        #########################ICU
        if accum_json[subject_id]['hadm_id'][hadm_id]['admin_info']['is_icu'] == 'Y' :
            t_icustay = icust_df[(icust_df['subject_id']==subject_id) & (icust_df['hadm_id']==hadm_id)]
            accum_json[subject_id]['hadm_id'][hadm_id]['icu_stay'] = {}
            accum_json[subject_id]['hadm_id'][hadm_id]['icu_stay']['los'] = t_icustay['los'].iloc[0]
            ######Vital Signs
            t_chartevent = chartevents_df.loc[(chartevents_df['subject_id']==subject_id) & (chartevents_df['hadm_id']==hadm_id)].sort_values("charttime")
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals'] = {}

            ###맥박 bpm
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['hr'] = []
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['hr_time'] = []
            t_vt = t_chartevent.loc[t_chartevent['itemid']==220045]
            for it, row in t_vt.iterrows() :
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['hr'].append(row['valuenum'])
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['hr_time'].append(row['charttime'])

            ###수축기 혈압 mmHg
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['sbp'] = []
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['sbp_time'] = []
            t_vt = t_chartevent.loc[t_chartevent['itemid']==220179]
            for it, row in t_vt.iterrows() :
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['sbp'].append(row['valuenum'])
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['sbp_time'].append(row['charttime'])

            ###이완기 혈압 mmHg
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['dbp'] = []
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['dbp_time'] = []
            t_vt = t_chartevent.loc[t_chartevent['itemid']==220180]
            for it, row in t_vt.iterrows() :
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['dbp'].append(row['valuenum'])
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['dbp_time'].append(row['charttime'])

            ###호흡 insp/min
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['rp'] = []
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['rp_time'] = []
            t_vt = t_chartevent.loc[t_chartevent['itemid']==220210]
            for it, row in t_vt.iterrows() :
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['rp'].append(row['valuenum'])
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['rp_time'].append(row['charttime'])

            ###산소 포화도 %
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['os'] = []
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['os_time'] = []
            t_vt = t_chartevent.loc[t_chartevent['itemid']==220277]
            for it, row in t_vt.iterrows() :
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['os'].append(row['valuenum'])
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['os_time'].append(row['charttime'])

            ###체온 섭씨
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['temp'] = []
            accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['temp_time'] = []
            t_vt = t_chartevent.loc[t_chartevent['itemid']==223761]
            for it, row in t_vt.iterrows() :
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['temp'].append(round((row['valuenum'] - 32) * 5/9, 2))
                accum_json[subject_id]['hadm_id'][hadm_id]['vitals']['temp_time'].append(row['charttime'])


            ######최근 근무자
            t_caregivers = t_chartevent.loc[t_chartevent['caregiver_id'].notna()]
            t_caregivers = t_caregivers.sort_values(["storetime", "charttime"])
            t_caregivers['next_caregiver'] = t_caregivers['caregiver_id'].shift(1, fill_value=0)
            t_caregivers = t_caregivers.loc[(t_caregivers['caregiver_id']!=t_caregivers['next_caregiver'])]
            accum_json[subject_id]['hadm_id'][hadm_id]['cargivers'] = []
            for it, row in t_caregivers.iterrows() :
                new_giver = {}
                new_giver['caregiver_id'] = row['caregiver_id']
                new_giver['chartstart'] = row['charttime']
                accum_json[subject_id]['hadm_id'][hadm_id]['cargivers'].append(new_giver)

        #####################################################
            #########일정 관련
            t_schedule = t_chartevent.drop_duplicates(subset= ["caregiver_id", "charttime", "category"])[['caregiver_id', 'charttime', 'category', 'label']]
            t_schedule = t_schedule.loc[t_schedule['caregiver_id'].notna()]
            #t_schedule['charttime'] = pd.to_datetime(t_schedule['charttime'])
            accum_json[subject_id]['hadm_id'][hadm_id]['schedule'] = []
            for it, row in t_schedule.iterrows() :
                if row['category'] == "Treatments" or row['category'] == "Care Plans" : 
                    new_schedule = {}
                    new_schedule['caregiver_id'] = row['caregiver_id']
                    new_schedule['charttime'] = row['charttime']
                    new_schedule['category'] = row['category'] 
                    new_schedule['label'] = row['label']
                    accum_json[subject_id]['hadm_id'][hadm_id]['schedule'].append(new_schedule)

            ######
            

        


In [27]:
t_caregivers = t_chartevent.loc[t_chartevent['caregiver_id'].notna()]
t_caregivers = t_caregivers.loc[t_caregivers['category']=="Treatments"]
t_caregivers = t_caregivers.sort_values(["storetime", "charttime"])
t_caregivers['caregiver_id'].unique()

array([19347., 14234.])

In [34]:
t_schedule.loc[(t_schedule['category']=="Treatments") | (t_schedule['category']=="Care Plans")]

,caregiver_id,charttime,category,label
999884,19347.0,2201-11-08 20:36:00,Treatments,Position
999908,19347.0,2201-11-08 20:37:00,Treatments,Oral Care
999287,14234.0,2201-11-10 08:00:00,Treatments,Anti Embolic Device Status
999476,14234.0,2201-11-10 11:00:00,Treatments,Head of Bed
999516,14234.0,2201-11-10 13:00:00,Treatments,Cough/Deep Breath
999518,14234.0,2201-11-10 13:31:00,Treatments,Head of Bed
999532,14234.0,2201-11-10 14:00:00,Treatments,Cough/Deep Breath
999587,14234.0,2201-11-10 14:27:00,Treatments,Therapeutic Bed
999604,14234.0,2201-11-10 14:28:00,Treatments,Oral Care
999642,14234.0,2201-11-10 16:00:00,Treatments,Oral Care


In [36]:
import json

# JSON 파일을 불러와서 사전으로 변환
with open('icu_data.json', 'w') as json_file:
    json.dump(accum_json, json_file, indent=4)


In [24]:
t_caregivers

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,...,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue,next_caregiver,next_charttime
999884,10027602,28166872,32391858,19347.0,2201-11-08 20:36:00,2201-11-08 20:36:00,224093,Left Side,NaN,NaN,...,Position,Position,chartevents,Treatments,NaN,Text,NaN,NaN,0.0,2201-11-10 08:00:00
999287,10027602,28166872,32391858,14234.0,2201-11-10 08:00:00,2201-11-10 08:00:00,225054,On,NaN,NaN,...,Anti Embolic Device Status,Anti Embolic Device Status,chartevents,Treatments,NaN,Text,NaN,NaN,19347.0,0


In [29]:
admissions_df.loc[admissions_df['subject_id']==10001217]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
11,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0
12,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaN,DIRECT EMER.,P99698,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,?,MARRIED,WHITE,NaN,NaN,0


In [46]:
omr_df.loc[admissions_df['subject_id']==10001217]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue


In [83]:
with open('./icu_data.json', 'r') as json_file:
    accum_json = json.load(json_file)

In [88]:
accum_json[int('10000032')]

KeyError: 10000032

In [92]:
accum_json.keys()

dict_keys(['10000032', '10000980', '10001217', '10001725', '10001884', '10002013', '10002155', '10002348', '10002428', '10002430', '10002443', '10002495', '10002760', '10002930', '10003019', '10003046', '10003400', '10003502', '10004113', '10004235', '10004401', '10004422', '10004457', '10004606', '10004720', '10004733', '10004764', '10005123', '10005348', '10005606', '10005817', '10005866', '10005909', '10006053', '10006131', '10006277', '10006580', '10006821', '10007058', '10007795', '10007818', '10007920', '10007928', '10008077', '10008100', '10008287', '10008454', '10008924', '10009035', '10009049', '10009628', '10009686', '10010058', '10010471', '10010867', '10011189', '10011365', '10011398', '10011427', '10011668', '10011938', '10012055', '10012206', '10012292', '10012438', '10012476', '10012552', '10012853', '10013015', '10013049', '10013310', '10013419', '10013569', '10013643', '10014078', '10014136', '10014179', '10014354', '10014610', '10014729', '10015272', '10015834', '1001